In [3]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

# Laden van de handmatig geannoteerde data
annotaties = pd.read_csv('Data/input/annotaties_handmatig.csv')

# Definiëren van het pad naar de afbeeldingen
image_dir = 'Data/input/images_resized'
annotated_dir = 'Data/input/images_resized_annotated'

In [8]:
def load_images_and_labels(image_dir, annotaties):
    images = []
    labels = []
    for index, row in annotaties.iterrows():
        img_path = os.path.join(image_dir, row['image_name'])
        image = cv2.imread(img_path)
        if image is not None:
            images.append(cv2.resize(image, (128, 128)))  # Resizing images to a fixed size
            label = 1 if row['label_name'] == 'Insect' else 0  # Assuming labels are 'Insect' and 'Geen Insect'
            labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_images_and_labels(annotated_dir, annotaties)
labels = to_categorical(labels, num_classes=2)  # Assuming binary classification (insect/not insect)

In [10]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

AttributeError: module 'keras.src.backend' has no attribute 'RandomGenerator'

In [ ]:
def active_learning_loop(model, unlabeled_images_dir, n_iterations=5, n_samples_per_iteration=100):
    unlabeled_images = [f for f in os.listdir(unlabeled_images_dir) if f.endswith('.jpg')]
    for iteration in range(n_iterations):
        uncertainties = []
        sampled_images = []
        
        for img_name in unlabeled_images:
            img_path = os.path.join(unlabeled_images_dir, img_name)
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.resize(image, (128, 128))
                prob = model.predict(np.expand_dims(image, axis=0))
                uncertainty = 1 - np.max(prob)
                uncertainties.append((img_name, uncertainty))
        
        uncertainties.sort(key=lambda x: x[1], reverse=True)
        sampled_images = [x[0] for x in uncertainties[:n_samples_per_iteration]]
        
        for img_name in sampled_images:
            img_path = os.path.join(unlabeled_images_dir, img_name)
            annotated_path = os.path.join(annotated_dir, img_name)
            os.rename(img_path, annotated_path)
            annotaties = annotaties.append({'image_name': img_name, 'label_name': int(input(f"Label voor {img_name} (1 voor insect, 0 voor geen insect): "))}, ignore_index=True)
        
        images, labels = load_images_and_labels(annotated_dir, annotaties)
        labels = to_categorical(labels, num_classes=2)
        model.fit(images, labels, epochs=5)


c:\Users\Gebruiker\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Gebruiker\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 2/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 3/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 4/10:   0%|          | 0/11 [00:00<?, ?it/s]

...

Epoch 6/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 7/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 8/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 9/10:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 10/10:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
final_images, final_labels = load_images_and_labels(annotated_dir, annotaties)
final_labels = to_categorical(final_labels, num_classes=2)
X_train, X_val, y_train, y_val = train_test_split(final_images, final_labels, test_size=0.2, random_state=42)

c:\Users\Gebruiker\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Gebruiker/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 37.8MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [ ]:
model = build_model()
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Laden van pool afbeeldingen:   0%|          | 0/39395 [00:00<?, ?it/s]

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {accuracy * 100:.2f}%")

Active learning iterations:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration 1/5


KeyboardInterrupt: 

In [ ]:
# Annotateer de resterende pool set
annotate_images(images_dir, model, 'annotaties_FasterRCNN.csv', device, num_workers=4)